In [1]:
import pandas as pd
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
right_hand = pd.read_csv("/home/ubuntu/Desktop/project/HandRight.csv")
def addAlone(p):
    r = right_hand 
    r = r.iloc[:len(p)-1]
    r = r.reset_index(drop=True)
    r.drop('Time', inplace=True,axis=1)
    p.set_index(' Frame ID',inplace=True)
    p = p.reset_index(drop=True)
    f = pd.concat([p,r],axis=1)
    f = f.drop([' Hand Type',' # hands',' Frame ID'],axis=1)
    return f
def addOther(p):
    r = p[p[' Hand Type']=='right']
    r.drop('Time', inplace=True,axis=1)
    l = p[p[' Hand Type']=='left']
    r.set_index(' Frame ID',inplace=True)
    l.set_index(' Frame ID',inplace=True)
    f = pd.DataFrame(pd.merge(l,r, how='left',on=' Frame ID'))
    f = f.drop([' Hand Type_x',' # hands_y' ,' Hand Type_y' ,' # hands_x'],axis=1)
    return f
def addDf(p,name,exp):
    p = p.iloc[701:]
    if 'Alone' not in exp:
        f = addOther(p)
    else:
        f = addAlone(p) 
    f.columns = ['Time', ' Position X', ' Position Y', ' Position Z', ' Velocity X', ' Velocity Y',
       ' Velocity Z', ' Pitch', ' Roll', ' Yaw', ' Wrist Pos X',
       ' Wrist Pos Y', ' Wrist Pos Z', ' Elbow pos X', ' Elbow Pos Y',
       ' Elbow Pos Z', ' Grab Strenth', ' Grab Angle', ' Pinch Strength', ' Position X r',
       ' Position Y r', ' Position Z r', ' Velocity X r', ' Velocity Y r',
       ' Velocity Z r', ' Pitch r', ' Roll r', ' Yaw r', ' Wrist Pos X r',
       ' Wrist Pos Y r', ' Wrist Pos Z r', ' Elbow pos X r', ' Elbow Pos Y r',
       ' Elbow Pos Z r', ' Grab Strenth r', ' Grab Angle r', ' Pinch Strength r']
    f.insert(loc=0, column='Name', value=name)
    f.insert(loc=1, column='Type', value=exp)
    return f

In [3]:
def loadData(pathW):
    df = pd.DataFrame(columns=[
        'Name',
        'Type',
        'Time',
        ' Position X',
        ' Position Y',
        ' Position Z',
        ' Velocity X',
        ' Velocity Y',
        ' Velocity Z',
        ' Pitch',
        ' Roll',
        ' Yaw',
        ' Wrist Pos X',
        ' Wrist Pos Y',
        ' Wrist Pos Z',
        ' Elbow pos X',
        ' Elbow Pos Y',
        ' Elbow Pos Z',
        ' Grab Strenth',
        ' Grab Angle',
        ' Pinch Strength',
        ' Position X r',
        ' Position Y r',
        ' Position Z r',
        ' Velocity X r',
        ' Velocity Y r',
        ' Velocity Z r',
        ' Pitch r',
        ' Roll r',
        ' Yaw r',
        ' Wrist Pos X r',
        ' Wrist Pos Y r',
        ' Wrist Pos Z r',
        ' Elbow pos X r',
        ' Elbow Pos Y r',
        ' Elbow Pos Z r',
        ' Grab Strenth r',
        ' Grab Angle r',
        ' Pinch Strength r',
        ])
    path = pathW
    import os
    for folder in os.listdir(path):
        c_A = 0
        c_Sp = 0
        c_Sy = 0
        for fu in os.listdir(path + folder):
            if 'Alone' in fu:
                c_A += 1
            elif 'Sync' in fu:
                c_Sy += 1
            else:
                c_Sp += 1
        for fu in os.listdir(path + folder):
            if 'Alone' in fu and c_A != 1:
                c_A -= 1
                continue
            elif 'Sync' in fu and c_Sy != 1:
                c_Sy -= 1
                continue
            elif 'Spontan' in fu and c_Sp != 1:
                c_Sp -= 1
                continue
            x = pd.read_csv(path + folder + '/' + fu)
            df = pd.concat([df, addDf(x, ''.join(filter(str.isalpha,
                           fu[:8])), ''.join(filter(str.isalpha,
                           fu[8:len(fu) - 3])))], sort=False)
    return df

In [4]:
train = loadData("/home/ubuntu/Desktop/project/Training/")
test = loadData("/home/ubuntu/Desktop/project/Validation/")

/home/ubuntu/Downloads/media/ubuntu/usbdata/tools/lib/python3.7/site-packages/pandas/core/frame.py:3940: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


In [7]:
train.dropna(inplace=True)
test.dropna(inplace=True)
x_train = train.drop(['Name','Type'], axis = 1)
y_train = train['Type']
x_test = test.drop(['Name','Type'], axis = 1)
y_test = test['Type']

In [8]:
pca = PCA(n_components=20)
principalComponents = pca.fit_transform(x_train)
principalDf = pd.DataFrame(data = principalComponents)
principalDf.reset_index(drop=True, inplace=True)
final = pd.concat([principalDf, y_train.reset_index(drop=True)],axis=1)
final.head()

,0,1,2,3,4,5,6,7,8,9,...,11,12,13,14,15,16,17,18,19,Type
0,-126.029619,255.735316,-128.474792,-7.683298,-55.381272,-54.286166,-39.122748,-81.807937,-10.885778,-34.730564,...,50.792450,19.032436,88.962199,-7.429238,-14.965027,-62.260742,-10.113337,-4.157756,-14.889303,Sync
1,-110.589600,197.633802,-127.564039,-4.553151,-55.880820,-47.254755,-53.201244,-81.022444,44.170016,-52.536468,...,58.109471,19.083079,85.971619,-6.289459,-15.800823,-59.828056,-11.950101,-3.563914,-13.249906,Sync
2,-91.281904,123.673533,-126.120669,-3.330491,-53.382365,-47.306340,-64.214961,-78.075529,71.430340,-61.583228,...,62.395333,18.097707,83.157500,-6.074114,-18.141449,-57.852525,-13.383369,-2.953356,-11.742912,Sync
3,-79.267303,113.571259,-126.125456,0.771326,-52.414248,-50.836364,-61.775561,-76.001535,55.924468,-61.098851,...,64.271740,15.105425,85.254817,-5.161345,-17.574002,-55.808368,-12.550713,-3.490000,-11.207754,Sync
4,-88.143056,162.603954,-127.049294,11.985858,-52.543777,-56.836761,-62.738692,-75.243780,39.967512,-60.492063,...,66.437286,12.459573,87.353226,-3.895345,-15.997047,-53.844897,-11.432732,-4.262184,-11.176919,Sync


In [ ]:
fig = plt.figure(figsize = (8,8))
ax = fig.add_subplot(1,1,1) 
ax.set_xlabel('Principal Component 1', fontsize = 15)
ax.set_ylabel('Principal Component 2', fontsize = 15)
ax.set_title('2 Component PCA', fontsize = 20)


targets = ['Alone', 'Sync', 'Spontan']
colors = ['r', 'g', 'b']
for target, color in zip(targets,colors):
    indicesToKeep = final['Type'] == target
    ax.scatter(final.loc[indicesToKeep, 1]
               , final.loc[indicesToKeep, 2]
               , c = color
               , s = 50)
ax.legend(targets)
ax.grid()

In [ ]:
scaler = StandardScaler()
# Fit on training set only.
scaler.fit(train_img)
# Apply transform to both the training set and the test set.
train_img = scaler.transform(X)